# Does the population of a city affect the winning percentage in sports?

This notebook uses 

**National Hockey League** :nhl.csv, 

**National Basketball Association** :nba.csv,

**National Football League** : nfl.csv,

**Major League Baseball** :mlb.csv

 A positive correlation (e.g., corr > 0) means that an increase in Population (2016 est.)[8] would also increase the values of winning percentage in sports

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


In [2]:
nhl = pd.read_csv("nhl.csv")

In [3]:
nhl.head()

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League
0,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,2018,NHL
1,Tampa Bay Lightning*,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL
2,Boston Bruins*,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL
3,Toronto Maple Leafs*,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL
4,Florida Panthers,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL


Load Cities in Wikipedia first table

In [4]:
cities=pd.read_html("Wikipedia.html")[1]

In [5]:
cities.head()

,Metropolitan area,Country,Pop.rank,Population (2016 est.)[8],B4,NFL,MLB,NBA,NHL,B6,MLS,CFL
0,New York City,United States,1,20153634,9,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,RangersIslandersDevils[note 3],11,Red BullsNew York City FC,—
1,Los Angeles,United States,2,13310447,8,RamsChargers[note 4],DodgersAngels,LakersClippers,KingsDucks,10,GalaxyLos Angeles FC[note 5],—
2,San Francisco Bay Area,United States,6,6657982,6,49ersRaiders[note 6],GiantsAthletics,Warriors,Sharks[note 7],7,Earthquakes,—
3,Chicago,United States,3,9512999,5,Bears[note 8],CubsWhite Sox,Bulls[note 9],Blackhawks,6,Fire,—
4,Dallas–Fort Worth,United States,4,7233323,4,Cowboys,Rangers,Mavericks,Stars,5,FC Dallas,—


In [6]:
cities.iloc[:1,[0,3,5,6,7,8]]

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,RangersIslandersDevils[note 3]


Filter only in 2018

In [7]:
nhl = nhl[nhl['year']==2018]

In [8]:
nhl.head()

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League
0,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,2018,NHL
1,Tampa Bay Lightning*,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL
2,Boston Bruins*,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL
3,Toronto Maple Leafs*,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL
4,Florida Panthers,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL


In [9]:
nhl.columns

Index(['team', 'GP', 'W', 'L', 'OL', 'PTS', 'PTS%', 'GF', 'GA', 'SRS', 'SOS',
       'RPt%', 'ROW', 'year', 'League'],
      dtype='object')

Data Cleaning

In [10]:
def remove_star(team):
    if "*" in team:
        return re.findall("([\w\s]*)\*",team)[0]
    else:
        return team


In [11]:
nhl['team'] = nhl['team'].apply(remove_star)

In [12]:
cities = cities[["Metropolitan area",'Population (2016 est.)[8]','NHL']]

In [13]:
def remove_bracket(team):
    if "[" in team:
        return re.findall("([\w\s]*)\[",team)[0]
    else:
        return team

In [14]:
cities['NHL'] = cities['NHL'].apply(remove_bracket) 

In [15]:
nhl.head()

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League
0,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,2018,NHL
1,Tampa Bay Lightning,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL
2,Boston Bruins,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL
3,Toronto Maple Leafs,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL
4,Florida Panthers,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL


Split team name with the area

In [16]:
nhl_df=nhl
nhl_df['team_only'] = nhl_df.team.apply(lambda x: x.rsplit(None, 1)[-1])

nhl_df.loc[3,'team_only']="Maple Leafs"
nhl_df.loc[5,'team_only']="Red Wings"
nhl_df.loc[13,'team_only']="Blue Jackets"
nhl_df.loc[27,'team_only']="Golden Knights"

nhl_df.loc[14,'team_only']="RangersIslandersDevils"
nhl_df.loc[16,'team_only']="RangersIslandersDevils"
nhl_df.loc[17,'team_only']="RangersIslandersDevils"

nhl_df.loc[28,'team_only']="KingsDucks"
nhl_df.loc[30,'team_only']="KingsDucks"

Remove Divisi pada Column 

In [17]:

cities.rename(columns={'Metropolitan area':'city','Population (2016 est.)[8]':'population','NHL':'team_only'},inplace=True)

nhl_df = nhl_df[~nhl_df.W.str.contains('Division')]

In [18]:


nhl_df.W = nhl_df.W.astype(float)
nhl_df.L = nhl_df.L.astype(float)

nhl_df = nhl_df.groupby(by='team_only')['W','L'].mean()

nhl_df.reset_index(inplace=True)

merged_df = pd.merge(nhl_df,cities,how='inner',on='team_only')

merged_df["W/L"] = merged_df.W / (merged_df.W + merged_df.L)


C:\Users\alifs\anaconda3\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
C:\Users\alifs\AppData\Local\Temp/ipykernel_21012/2108936596.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  nhl_df = nhl_df.groupby(by='team_only')['W','L'].mean()


In [19]:
nhl_df.head()

,team_only,W,L
0,Avalanche,43.0,30.0
1,Blackhawks,33.0,39.0
2,Blue Jackets,45.0,30.0
3,Blues,44.0,32.0
4,Bruins,50.0,20.0


Changing data type population and W/L

In [20]:
population = pd.to_numeric(merged_df['population'],errors='coerce') 
win_loss = pd.to_numeric(merged_df['W/L'],errors='coerce') 

In [21]:
corrnhl=stats.pearsonr(population, win_loss)[0]

In [22]:
corrnhl

0.012308996455744278

# MLB

In [23]:
mlb=pd.read_csv("mlb.csv")
cities=pd.read_html("wikipedia.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

In [24]:
mlb.head()

,team,W,L,W-L%,GB,year,League
0,Boston Red Sox,108,54,0.667,--,2018,MLB
1,New York Yankees,100,62,0.617,8.0,2018,MLB
2,Tampa Bay Rays,90,72,0.556,18.0,2018,MLB
3,Toronto Blue Jays,73,89,0.451,35.0,2018,MLB
4,Baltimore Orioles,47,115,0.290,61.0,2018,MLB


In [25]:
cities.drop([24,25,26,28,29,30,31,32,33,34,35,36,37,38,39,41,42,43,44,45,46,47,48,49,50],0,inplace=True)

C:\Users\alifs\AppData\Local\Temp/ipykernel_21012/3445985063.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  cities.drop([24,25,26,28,29,30,31,32,33,34,35,36,37,38,39,41,42,43,44,45,46,47,48,49,50],0,inplace=True)


In [26]:
cities.head()

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,RangersIslandersDevils[note 3]
1,Los Angeles,13310447,RamsChargers[note 4],DodgersAngels,LakersClippers,KingsDucks
2,San Francisco Bay Area,6657982,49ersRaiders[note 6],GiantsAthletics,Warriors,Sharks[note 7]
3,Chicago,9512999,Bears[note 8],CubsWhite Sox,Bulls[note 9],Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars


In [27]:
mlb=mlb.head(30)

In [28]:
mlb['team_ville'] = mlb['team']
mlb['team_ville'] = mlb['team_ville'].map({'Boston Red Sox':'Boston',
'New York Yankees':'New York City',
'Tampa Bay Rays':'Tampa Bay Area',
'Toronto Blue Jays':'Toronto',
'Baltimore Orioles':'Baltimore',
'Cleveland Indians':'Cleveland',
'Minnesota Twins':'Minneapolis–Saint Paul',
'Detroit Tigers':'Detroit',
'Chicago White Sox':'Chicago',
'Kansas City Royals':'Kansas City',
'Houston Astros':'Houston',
'Oakland Athletics':'San Francisco Bay Area',
'Seattle Mariners':'Seattle',
'Los Angeles Angels':'Los Angeles',
'Texas Rangers':'Dallas–Fort Worth',
'Atlanta Braves':'Atlanta',
'Washington Nationals':'Washington, D.C.',
'Philadelphia Phillies':'Philadelphia',
'New York Mets':'New York City',
'Miami Marlins':'Miami–Fort Lauderdale',
'Milwaukee Brewers':'Milwaukee',
'Chicago Cubs':'Chicago',
'St. Louis Cardinals':'St. Louis',
'Pittsburgh Pirates':'Pittsburgh',
'Cincinnati Reds':'Cincinnati',
'Los Angeles Dodgers':'Los Angeles',
'Colorado Rockies':'Denver',
'Arizona Diamondbacks':'Phoenix',
'San Francisco Giants':'San Francisco Bay Area',
'San Diego Padres':'San Diego'})

In [29]:
m_mlb=pd.merge(mlb,cities, left_on= "team_ville", right_on= "Metropolitan area")

In [30]:
m_mlb.head()

,team,W,L,W-L%,GB,year,League,team_ville,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,Boston Red Sox,108,54,0.667,--,2018,MLB,Boston,Boston,4794447,Patriots[note 14],Red Sox[note 15],Celtics,Bruins
1,New York Yankees,100,62,0.617,8.0,2018,MLB,New York City,New York City,20153634,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,RangersIslandersDevils[note 3]
2,New York Mets,77,85,0.475,13.0,2018,MLB,New York City,New York City,20153634,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,RangersIslandersDevils[note 3]
3,Tampa Bay Rays,90,72,0.556,18.0,2018,MLB,Tampa Bay Area,Tampa Bay Area,3032171,Buccaneers,Rays,—,Lightning
4,Toronto Blue Jays,73,89,0.451,35.0,2018,MLB,Toronto,Toronto,5928040,— [note 22],Blue Jays,Raptors[note 23],Maple Leafs


In [31]:
m_mlb.W= pd.to_numeric(m_mlb.W)
m_mlb.L= pd.to_numeric(m_mlb.L)
m_mlb['Population (2016 est.)[8]'] = pd.to_numeric(m_mlb['Population (2016 est.)[8]'])
m_mlb = m_mlb[['team','W','L','Metropolitan area','Population (2016 est.)[8]']]

In [32]:
m_mlb['W/L'] = m_mlb.W/(m_mlb.L+m_mlb.W)

In [33]:
m_mlb['W/L'].head()

0    0.666667
1    0.617284
2    0.475309
3    0.555556
4    0.450617
Name: W/L, dtype: float64

In [34]:
m_mlb=m_mlb.groupby('Metropolitan area').mean().reset_index()

In [35]:
population=m_mlb['Population (2016 est.)[8]']
win_loss=m_mlb['W/L']

In [36]:
mlb=m_mlb

In [37]:
corrmlb =stats.pearsonr(population, win_loss)[0]

In [38]:
corrmlb

0.15027698302669307

# NFL

In [39]:
nfl=pd.read_csv("nfl.csv")
cities=pd.read_html("wikipedia.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

In [40]:
nfl.head(5)

,DSRS,L,League,MoV,OSRS,PA,PD,PF,SRS,SoS,T,W,W-L%,team,year
0,AFC East,AFC East,NFL,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,2018
1,2.1,5,NFL,6.9,3.1,325,111,436,5.2,-1.8,0,11,.688,New England Patriots*,2018
2,-5.2,9,NFL,-7.1,-3.6,433,-114,319,-8.8,-1.7,0,7,.438,Miami Dolphins,2018
3,-0.6,10,NFL,-6.6,-6.3,374,-105,269,-6.9,-0.3,0,6,.375,Buffalo Bills,2018
4,-5.9,12,NFL,-6.8,-2.0,441,-108,333,-7.8,-1.1,0,4,.250,New York Jets,2018


In [41]:
nfl.drop([0,5,10,15,20,25,30,35],0,inplace=True)

C:\Users\alifs\AppData\Local\Temp/ipykernel_21012/1690286940.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  nfl.drop([0,5,10,15,20,25,30,35],0,inplace=True)


In [42]:
nfl.head()

,DSRS,L,League,MoV,OSRS,PA,PD,PF,SRS,SoS,T,W,W-L%,team,year
1,2.1,5,NFL,6.9,3.1,325,111,436,5.2,-1.8,0,11,.688,New England Patriots*,2018
2,-5.2,9,NFL,-7.1,-3.6,433,-114,319,-8.8,-1.7,0,7,.438,Miami Dolphins,2018
3,-0.6,10,NFL,-6.6,-6.3,374,-105,269,-6.9,-0.3,0,6,.375,Buffalo Bills,2018
4,-5.9,12,NFL,-6.8,-2.0,441,-108,333,-7.8,-1.1,0,4,.250,New York Jets,2018
6,6.4,6,NFL,6.4,0.6,287,102,389,7.0,0.6,0,10,.625,Baltimore Ravens*,2018


In [43]:
cities.head()

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,RangersIslandersDevils[note 3]
1,Los Angeles,13310447,RamsChargers[note 4],DodgersAngels,LakersClippers,KingsDucks
2,San Francisco Bay Area,6657982,49ersRaiders[note 6],GiantsAthletics,Warriors,Sharks[note 7]
3,Chicago,9512999,Bears[note 8],CubsWhite Sox,Bulls[note 9],Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars


In [44]:
cities.drop([13,22,27,30,31,32,33,34,35,36,37,38,39,40,41,42,43,45,46,47,49,50],0,inplace=True)

C:\Users\alifs\AppData\Local\Temp/ipykernel_21012/772719823.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  cities.drop([13,22,27,30,31,32,33,34,35,36,37,38,39,40,41,42,43,45,46,47,49,50],0,inplace=True)


In [45]:
l1 = []
for i in nfl['team']:
    #i=i.rstrip()
    i=i.split('*')
    l1.append(i[0])
nfl['team'] = l1

l2 = []
for i in nfl['team']:
    i=i.split('+')
    l2.append(i[0])
nfl['team'] = l2

nfl = nfl.head(32)

In [46]:
nfl['team_ville'] = nfl['team']
nfl['team_ville'] = nfl['team_ville'].map({'New England Patriots':'Boston',
 'Miami Dolphins':'Miami–Fort Lauderdale',
 'Buffalo Bills':'Buffalo',
 'New York Jets':'New York City',
 'Baltimore Ravens':'Baltimore',
 'Pittsburgh Steelers':'Pittsburgh',
 'Cleveland Browns':'Cleveland',
 'Cincinnati Bengals':'Cincinnati',
 'Houston Texans':'Houston',
 'Indianapolis Colts':'Indianapolis',
 'Tennessee Titans':'Nashville',
 'Jacksonville Jaguars':'Jacksonville',
 'Kansas City Chiefs':'Kansas City',
 'Los Angeles Chargers':'Los Angeles',
 'Denver Broncos':'Denver',
 'Oakland Raiders':'San Francisco Bay Area',
 'Dallas Cowboys':'Dallas–Fort Worth',
 'Philadelphia Eagles':'Philadelphia',
 'Washington Redskins':'Washington, D.C.',
 'New York Giants':'New York City',
 'Chicago Bears':'Chicago',
 'Minnesota Vikings':'Minneapolis–Saint Paul',
 'Green Bay Packers':'Green Bay',
 'Detroit Lions':'Detroit',
 'New Orleans Saints':'New Orleans',
 'Carolina Panthers':'Charlotte',
 'Atlanta Falcons':'Atlanta',
 'Tampa Bay Buccaneers':'Tampa Bay Area',
 'Los Angeles Rams':'Los Angeles',
 'Seattle Seahawks':'Seattle',
 'San Francisco 49ers':'San Francisco Bay Area',
 'Arizona Cardinals':'Phoenix'}) 

In [47]:
m_nfl = pd.merge(nfl,cities, left_on= "team_ville", right_on= "Metropolitan area")


m_nfl['W'] = pd.to_numeric(m_nfl['W'])
m_nfl['L'] = pd.to_numeric(m_nfl['L'])
m_nfl['Population (2016 est.)[8]'] = pd.to_numeric(m_nfl['Population (2016 est.)[8]'])

m_nfl  = m_nfl[['team','W','L','Metropolitan area','Population (2016 est.)[8]']]
m_nfl['W/L'] = m_nfl['W']/(m_nfl['L']+m_nfl['W'])
m_nfl  = m_nfl.groupby('Metropolitan area').mean().reset_index()
population = m_nfl['Population (2016 est.)[8]'] # pass in metropolitan area population from cities
win_loss = m_nfl ['W/L']
nfl=m_nfl

In [48]:
corrnfl = stats.pearsonr(population,win_loss)[0]
corrnfl


0.004922112149349456

# NBA

In [49]:
nba = pd.read_csv("nba.csv")
cities = pd.read_html("Wikipedia.html")[1]
cities = cities.iloc[:-1,[0,3,5,6,7,8]]

In [50]:
nba.head()

,team,W,L,W/L%,GB,PS/G,PA/G,SRS,year,League
0,Toronto Raptors* (1),59,23,0.72,—,111.7,103.9,7.29,2018,NBA
1,Boston Celtics* (2),55,27,0.6709999999999999,4.0,104.0,100.4,3.23,2018,NBA
2,Philadelphia 76ers* (3),52,30,0.634,7.0,109.8,105.3,4.3,2018,NBA
3,Cleveland Cavaliers* (4),50,32,0.61,9.0,110.9,109.9,0.59,2018,NBA
4,Indiana Pacers* (5),48,34,0.585,11.0,105.6,104.2,1.18,2018,NBA


In [51]:
nba = nba[nba.year==2018]

In [52]:
nba['team'] = nba.team.apply(lambda team: re.findall("([\w\s]*)\*", team)[0] if '*' in team else team)

nba['team'] = nba.team.apply(lambda team: re.findall("([\w\s]*)\(", team)[0] if '(' in team else team)

cities['NBA'] = cities.NBA.apply(lambda team: re.findall("([\w\s]*)\[", team)[0] if '[' in team else team)


In [53]:
nba.head()

,team,W,L,W/L%,GB,PS/G,PA/G,SRS,year,League
0,Toronto Raptors,59,23,0.72,—,111.7,103.9,7.29,2018,NBA
1,Boston Celtics,55,27,0.6709999999999999,4.0,104.0,100.4,3.23,2018,NBA
2,Philadelphia 76ers,52,30,0.634,7.0,109.8,105.3,4.3,2018,NBA
3,Cleveland Cavaliers,50,32,0.61,9.0,110.9,109.9,0.59,2018,NBA
4,Indiana Pacers,48,34,0.585,11.0,105.6,104.2,1.18,2018,NBA


In [54]:
l1 = []
for i in nba['team']:
    #i=i.rstrip()
    i=i.split('*')
    l1.append(i[0])
nba['team'] = l1

l2 = []
for i in nba['team']:
    i=i.split('(')
    l2.append(i[0])
nba['team'] = l2

l3 = []
for i in nba['team']:
    i=i.rstrip()
    l3.append(i)
nba['team'] = l3


In [55]:
nba= nba.head(30)

nba['team_ville'] = nba['team']
nba['team_ville'] = nba['team_ville'].map({'Toronto Raptors':'Toronto',
 'Boston Celtics':'Boston',
 'Philadelphia 76ers':'Philadelphia',
 'Cleveland Cavaliers':'Cleveland',
 'Indiana Pacers':'Indianapolis',
 'Miami Heat':'Miami–Fort Lauderdale',
 'Milwaukee Bucks':'Milwaukee',
 'Washington Wizards':'Washington, D.C.',
 'Detroit Pistons':'Detroit',
 'Charlotte Hornets':'Charlotte',
 'New York Knicks':'New York City',
 'Brooklyn Nets':'New York City',
 'Chicago Bulls':'Chicago',
 'Orlando Magic':'Orlando',
 'Atlanta Hawks':'Atlanta',
 'Houston Rockets':'Houston',
 'Golden State Warriors':'San Francisco Bay Area',
 'Portland Trail Blazers':'Portland',
 'Oklahoma City Thunder':'Oklahoma City',
 'Utah Jazz':'Salt Lake City',
 'New Orleans Pelicans':'New Orleans',
 'San Antonio Spurs':'San Antonio',
 'Minnesota Timberwolves':'Minneapolis–Saint Paul',
 'Denver Nuggets':'Denver',
 'Los Angeles Clippers':'Los Angeles',
 'Los Angeles Lakers':'Los Angeles',
 'Sacramento Kings':'Sacramento',
 'Dallas Mavericks':'Dallas–Fort Worth',
 'Memphis Grizzlies':'Memphis',
 'Phoenix Suns':'Phoenix'})

In [56]:
nba = pd.merge(nba,cities, left_on= "team_ville", right_on= "Metropolitan area")

In [57]:
nba.head()

,team,W,L,W/L%,GB,PS/G,PA/G,SRS,year,League,team_ville,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,Toronto Raptors,59,23,0.72,—,111.7,103.9,7.29,2018,NBA,Toronto,Toronto,5928040,— [note 22],Blue Jays,Raptors,Maple Leafs
1,Boston Celtics,55,27,0.6709999999999999,4.0,104.0,100.4,3.23,2018,NBA,Boston,Boston,4794447,Patriots[note 14],Red Sox[note 15],Celtics,Bruins
2,Philadelphia 76ers,52,30,0.634,7.0,109.8,105.3,4.3,2018,NBA,Philadelphia,Philadelphia,6070500,Eagles,Phillies[note 12],76ers,Flyers[note 13]
3,Cleveland Cavaliers,50,32,0.61,9.0,110.9,109.9,0.59,2018,NBA,Cleveland,Cleveland,2055612,Browns[note 29],Indians[note 30],Cavaliers,[note 32]
4,Indiana Pacers,48,34,0.585,11.0,105.6,104.2,1.18,2018,NBA,Indianapolis,Indianapolis,2004230,Colts,[note 50],Pacers,—


In [58]:
nba['Population (2016 est.)[8]'] = pd.to_numeric(nba['Population (2016 est.)[8]'])
nba['W'] = pd.to_numeric(nba['W'])
nba['L'] = pd.to_numeric(nba['L'])


In [59]:
nba= nba[['team','W','L','Metropolitan area','Population (2016 est.)[8]']]

In [60]:
nba.head()

,team,W,L,Metropolitan area,Population (2016 est.)[8]
0,Toronto Raptors,59,23,Toronto,5928040
1,Boston Celtics,55,27,Boston,4794447
2,Philadelphia 76ers,52,30,Philadelphia,6070500
3,Cleveland Cavaliers,50,32,Cleveland,2055612
4,Indiana Pacers,48,34,Indianapolis,2004230


In [61]:
nba['W/L'] =nba.W / (nba.W + nba.L)
nba = nba.groupby('Metropolitan area').mean().reset_index()
population = nba['Population (2016 est.)[8]']
win_loss = nba['W/L']
nba.head()

,Metropolitan area,W,L,Population (2016 est.)[8],W/L
0,Atlanta,24.0,58.0,5789700.0,0.292683
1,Boston,55.0,27.0,4794447.0,0.670732
2,Charlotte,36.0,46.0,2474314.0,0.439024
3,Chicago,27.0,55.0,9512999.0,0.329268
4,Cleveland,50.0,32.0,2055612.0,0.609756


In [62]:
corrnba = stats.pearsonr(population,win_loss)[0]
corrnba


-0.17657160252844611

In [63]:
nba.head()

,Metropolitan area,W,L,Population (2016 est.)[8],W/L
0,Atlanta,24.0,58.0,5789700.0,0.292683
1,Boston,55.0,27.0,4794447.0,0.670732
2,Charlotte,36.0,46.0,2474314.0,0.439024
3,Chicago,27.0,55.0,9512999.0,0.329268
4,Cleveland,50.0,32.0,2055612.0,0.609756


In [64]:
mlb.head()

,Metropolitan area,W,L,Population (2016 est.)[8],W/L
0,Atlanta,90.0,72.0,5789700.0,0.555556
1,Baltimore,47.0,115.0,2798886.0,0.290123
2,Boston,108.0,54.0,4794447.0,0.666667
3,Chicago,78.5,84.0,9512999.0,0.482769
4,Cincinnati,67.0,95.0,2165139.0,0.413580


In [65]:
nfl.head()

,Metropolitan area,W,L,Population (2016 est.)[8],W/L
0,Atlanta,7.0,9.0,5789700.0,0.4375
1,Baltimore,10.0,6.0,2798886.0,0.6250
2,Boston,11.0,5.0,4794447.0,0.6875
3,Buffalo,6.0,10.0,1132804.0,0.3750
4,Charlotte,7.0,9.0,2474314.0,0.4375


In [66]:
nhl_df=pd.read_csv("nhl.csv")
cities=pd.read_html("Wikipedia.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

nhl_df.drop([0,9,18,26],0,inplace=True)
cities.drop([14,15,18,19,20,21,23,24,25,27,28,32,33,38,40,41,42,44,45,46,48,49,50],0,inplace=True)

l= []
for i in cities['NHL']:
    i=i.split('[')
    l.append(i[0])
cities['NHL'] = l

li = []
for i in nhl_df['team']:
    i = re.findall("[^*]+", i)
    li.append(i[0])
nhl_df['team'] = li

nhl_df = nhl_df.head(31)

nhl_df['team_ville'] = nhl_df['team']
nhl_df['team_ville'] = nhl_df['team_ville'].map({'Tampa Bay Lightning':'Tampa Bay Area',
 'Boston Bruins':'Boston',
 'Toronto Maple Leafs':'Toronto',
 'Florida Panthers':'Miami–Fort Lauderdale',
 'Detroit Red Wings':'Detroit',
 'Montreal Canadiens':'Montreal',
 'Ottawa Senators':'Ottawa',
 'Buffalo Sabres':'Buffalo',
 'Washington Capitals':'Washington, D.C.',
 'Pittsburgh Penguins':'Pittsburgh',
 'Philadelphia Flyers':'Philadelphia',
 'Columbus Blue Jackets':'Columbus',
 'New Jersey Devils':'New York City',
 'Carolina Hurricanes':'Raleigh',
 'New York Islanders':'New York City',
 'New York Rangers':'New York City',
 'Nashville Predators':'Nashville',
 'Winnipeg Jets':'Winnipeg',
 'Minnesota Wild':'Minneapolis–Saint Paul',
 'Colorado Avalanche':'Denver',
 'St. Louis Blues':'St. Louis',
 'Dallas Stars':'Dallas–Fort Worth',
 'Chicago Blackhawks':'Chicago',
 'Vegas Golden Knights':'Las Vegas',
 'Anaheim Ducks':'Los Angeles',
 'San Jose Sharks':'San Francisco Bay Area',
 'Los Angeles Kings':'Los Angeles',
 'Calgary Flames':'Calgary',
 'Edmonton Oilers':'Edmonton',
 'Vancouver Canucks':'Vancouver',
 'Arizona Coyotes':'Phoenix'})
df = pd.merge(nhl_df,cities, left_on= "team_ville", right_on= "Metropolitan area")

df['W'] = pd.to_numeric(df['W'])
df['L'] = pd.to_numeric(df['L'])
df['Population (2016 est.)[8]'] = pd.to_numeric(df['Population (2016 est.)[8]'])


df = df[['team','W','L','Metropolitan area','Population (2016 est.)[8]']]

df['W/L'] = df['W']/(df['L']+df['W'])

df = df.groupby('Metropolitan area').mean().reset_index()

nhl= df

C:\Users\alifs\AppData\Local\Temp/ipykernel_21012/3292145387.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  nhl_df.drop([0,9,18,26],0,inplace=True)
C:\Users\alifs\AppData\Local\Temp/ipykernel_21012/3292145387.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  cities.drop([14,15,18,19,20,21,23,24,25,27,28,32,33,38,40,41,42,44,45,46,48,49,50],0,inplace=True)


In [67]:
df.head()

,Metropolitan area,W,L,Population (2016 est.)[8],W/L
0,Boston,50.0,20.0,4794447.0,0.714286
1,Buffalo,25.0,45.0,1132804.0,0.357143
2,Calgary,37.0,35.0,1392609.0,0.513889
3,Chicago,33.0,39.0,9512999.0,0.458333
4,Columbus,45.0,30.0,2041520.0,0.600000


# Calculate Pval

**null null hypothesis**: that an area has two sports teams in different sports, those teams will perform the same within their respective sports

Are there any sports where we can reject the null hypothesis? 

In [68]:
nba_nfl = pd.merge(nba,nfl, on='Metropolitan area')
pval_nba_nfl = stats.ttest_rel(nba_nfl['W/L_x'],nba_nfl['W/L_y'])[1]
nba_nhl = pd.merge(nba,nhl, on='Metropolitan area')
pval_nba_nhl = stats.ttest_rel(nba_nhl['W/L_x'],nba_nhl['W/L_y'])[1]
mlb_nfl = pd.merge(mlb,nfl, on='Metropolitan area')
pval_mlb_nfl = stats.ttest_rel(mlb_nfl['W/L_x'],mlb_nfl['W/L_y'])[1]
mlb_nhl = pd.merge(mlb,nhl, on='Metropolitan area')
pval_mlb_nhl = stats.ttest_rel(mlb_nhl['W/L_x'],mlb_nhl['W/L_y'])[1]
mlb_nba = pd.merge(mlb,nba, on='Metropolitan area')
pval_mlb_nba = stats.ttest_rel(mlb_nba['W/L_x'],mlb_nba['W/L_y'])[1]
nhl_nfl = pd.merge(nhl,nfl, on='Metropolitan area')
pval_nhl_nfl = stats.ttest_rel(nhl_nfl['W/L_x'],nhl_nfl['W/L_y'])[1]

pv = {'NFL': {"NFL": np.nan, 'NBA': pval_nba_nfl, 'NHL': pval_nhl_nfl, 'MLB': pval_mlb_nfl},
   'NBA': {"NFL": pval_nba_nfl, 'NBA': np.nan, 'NHL': pval_nba_nhl, 'MLB': pval_mlb_nba},
   'NHL': {"NFL": pval_nhl_nfl, 'NBA': pval_nba_nhl, 'NHL': np.nan, 'MLB': pval_mlb_nhl},
   'MLB': {"NFL": pval_mlb_nfl, 'NBA': pval_mlb_nba, 'NHL': pval_mlb_nhl, 'MLB': np.nan}
  }



p_values = pd.DataFrame(pv)


p_values


,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030883,0.802069
NBA,0.941792,NaN,0.022297,0.950540
NHL,0.030883,0.022297,NaN,0.000708
MLB,0.802069,0.950540,0.000708,NaN


In [69]:
nba_nfl

,Metropolitan area,W_x,L_x,Population (2016 est.)[8]_x,W/L_x,W_y,L_y,Population (2016 est.)[8]_y,W/L_y
0,Atlanta,24.0,58.0,5789700.0,0.292683,7.0,9.0,5789700.0,0.437500
1,Boston,55.0,27.0,4794447.0,0.670732,11.0,5.0,4794447.0,0.687500
2,Charlotte,36.0,46.0,2474314.0,0.439024,7.0,9.0,2474314.0,0.437500
3,Chicago,27.0,55.0,9512999.0,0.329268,12.0,4.0,9512999.0,0.750000
4,Cleveland,50.0,32.0,2055612.0,0.609756,7.0,8.0,2055612.0,0.466667
5,Dallas–Fort Worth,24.0,58.0,7233323.0,0.292683,10.0,6.0,7233323.0,0.625000
6,Denver,46.0,36.0,2853077.0,0.560976,6.0,10.0,2853077.0,0.375000
7,Detroit,39.0,43.0,4297617.0,0.475610,6.0,10.0,4297617.0,0.375000
8,Houston,65.0,17.0,6772470.0,0.792683,11.0,5.0,6772470.0,0.687500
9,Indianapolis,48.0,34.0,2004230.0,0.585366,10.0,6.0,2004230.0,0.625000


**NHL** is s one of sport can reject the null hypothesis ,it means a team in a city that has two sports teams **would not perform the same**